In [22]:
from selenium import webdriver
from lxml import etree
import time
import re
from selenium.webdriver.chrome.options import Options
import requests
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import WebDriverWait  
from selenium.webdriver.support import expected_conditions as EC  
import pandas as pd
from datetime import datetime
import os
import json

In [2]:
# chromedriver的绝对路径
driver_path = r'C:\Users\ZHOU-JC\Desktop\chromedriver.exe'
#设置无头 
# chrome_options = Options()
# chrome_options.add_argument('--headless')
# driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)
chrome_options = Options()
# chrome_options.add_argument("disable-web-security")

# 初始化一个driver，并且指定chromedriver的路径
driver = webdriver.Chrome(executable_path=driver_path, options =chrome_options)

try:
    driver.get("https://gz.zu.fang.com/")
except:
    # 当页面加载时间超过设定时间，通过js来stop，即可执行后续动作
    driver.execute_script("window.stop()")
# 请求网页

In [12]:

'''
广州 = [
https://gz.zu.fang.com/house-a080/  #增城 45页
https://gz.zu.fang.com/house-a078/g21/ #番禺 一居 29页
https://gz.zu.fang.com/house-a078/g22/ #番禺 二居 54页
https://gz.zu.fang.com/house-a078/g23/ #番禺 三居 66页
https://gz.zu.fang.com/house-a078/g24/ #番禺 四居 21页
https://gz.zu.fang.com/house-a078/g299/ #番禺 四居以上 13页
https://gz.zu.fang.com/house-a084/  #南沙 40页
https://gz.zu.fang.com/house-a0639/ #花都 26页
https://gz.zu.fang.com/house-a076/g21/ #白云 一居 23页
https://gz.zu.fang.com/house-a076/g22/ #白云 二居 40页
https://gz.zu.fang.com/house-a076/g23/ #白云 三居 39页
https://gz.zu.fangh.com/house-a076/g24/ #白云 四居 9页
https://gz.zu.fang.com/house-a076/g299/ #白云 四居以上 3页
https://gz.zu.fang.com/house-a074/g21/ #海珠 一居 36页
https://gz.zu.fang.com/house-a074/g22/ #海珠 二居 83页
https://gz.zu.fang.com/house-a074/g23/ #海珠 三居 58页
https://gz.zu.fang.com/house-a074/g24/ #海珠 四居 19页 
https://gz.zu.fang.com/house-a074/g299/ #海珠 四居以上 7页
https://gz.zu.fang.com/house-a072/g21/ #越秀 一居 46页 
https://gz.zu.fang.com/house-a072/g22/ #越秀 二居 100页
https://gz.zu.fang.com/house-a072/g23/ #越秀 三居 65页
https://gz.zu.fang.com/house-a072/g24/ #越秀 四居 7页
https://gz.zu.fang.com/house-a072/g299/ #越秀 四居以上 4页
https://gz.zu.fang.com/house-a071/ #荔湾 72页 
https://gz.zu.fang.com/house-a073/g21/ #天河 一居 100页
https://gz.zu.fang.com/house-a073/g22/ #天河 二居 100页
https://gz.zu.fang.com/house-a073/g23/ #天河 三居 100页
https://gz.zu.fang.com/house-a073/g24/ #天河 四居 33页
https://gz.zu.fang.com/house-a073/g299/ #天河 四居以上 10页
https://gz.zu.fang.com/house-a079/ #从化 3页
https://gz.zu.fang.com/house-a075/ #黄埔 86页
https://gz.zu.fang.com/house-a015882/ #广州周边 10页
]
'''

'\n广州 = [\nhttps://gz.zu.fang.com/house-a080/  #增城 45页\nhttps://gz.zu.fang.com/house-a078/g21/ #番禺 一居 29页\nhttps://gz.zu.fang.com/house-a078/g22/ #番禺 二居 54页\nhttps://gz.zu.fang.com/house-a078/g23/ #番禺 三居 66页\nhttps://gz.zu.fang.com/house-a078/g24/ #番禺 四居 21页\nhttps://gz.zu.fang.com/house-a078/g299/ #番禺 四居以上 13页\nhttps://gz.zu.fang.com/house-a084/  #南沙 40页\nhttps://gz.zu.fang.com/house-a0639/ #花都 26页\nhttps://gz.zu.fang.com/house-a076/g21/ #白云 一居 23页\nhttps://gz.zu.fang.com/house-a076/g22/ #白云 二居 40页\nhttps://gz.zu.fang.com/house-a076/g23/ #白云 三居 39页\nhttps://gz.zu.fang.com/house-a076/g24/ #白云 四居 9页\nhttps://gz.zu.fang.com/house-a076/g299/ #白云 四居以上 3页\nhttps://gz.zu.fang.com/house-a074/g21/ #海珠 一居 36页\nhttps://gz.zu.fang.com/house-a074/g22/ #海珠 二居 83页\nhttps://gz.zu.fang.com/house-a074/g23/ #海珠 三居 58页\nhttps://gz.zu.fang.com/house-a074/g24/ #海珠 四居 19页 \nhttps://gz.zu.fang.com/house-a074/g299/ #海珠 四居以上 7页\nhttps://gz.zu.fang.com/house-a072/g21/ #越秀 一居 46页 \nhttps://gz.zu.fang.com/house-a

# 增城 (45页)

In [7]:
driver.switch_to.window(driver.window_handles[0])
driver.execute_script("window.open('https://gz.zu.fang.com/house-a080/')")
driver.switch_to.window(driver.window_handles[-1])

In [10]:
driver.switch_to.window(driver.window_handles[1])  #跳回列表页

Infos = []
next_page = driver.find_element_by_partial_link_text("下一页")
pages_detail = driver.find_elements_by_xpath("//p[@class='title']/a")
while next_page:
    print('当前爬的索引页为', driver.current_url)
    for page_detail in pages_detail:
        page_detail.click()
        driver.switch_to.window(driver.window_handles[-1])  #跳到详情页
        WebDriverWait(driver, 0.5).until(EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, '装修')))   
        source = driver.page_source
        try: 
            Info = parse_detail_page(source)
        except:
            pass 
        Info['网站'] = driver.current_url
        print(Info)
        
        #保存入json文件
        jsonbj = json.dumps(Info)
        self.json.write("\n"+jsonbj)
        
        Infos.append(Info)
        driver.close()
        driver.switch_to.window(driver.window_handles[1])  #跳回列表页
    next_page.click()
    driver.switch_to.window(driver.window_handles[-1])  #跳回新的索引页  
    try:
        next_page = driver.find_element_by_partial_link_text("下一页")
        pages_detail = driver.find_elements_by_xpath("//p[@class='title']/a")
    except:
        next_page = None
        
        print('爬虫完毕')

当前爬的索引页为 https://gz.zu.fang.com/house-a080/
{'标题': '新塘地铁口豪装大三房，房子比照片更漂亮。仅租2100元，实价', '价格_详情': ' 2250 元/月（押二付一） ', '价格': '2250', '出租方式': '整租', '朝向': '东南', '户型': '3室2厅2卫', '楼层': '中层 楼层（共23层）', '建筑面积': '155平米', '装修': '豪华装修', '小区': ['现代城市花园', '增城', '新塘北'], '地址': ['距13号线新塘站约333米'], '网站': 'https://gz.zu.fang.com/chuzu/3_238591073_1.htm?channel=1,2'}
{'标题': '翡翠绿洲铂金精装三房，业主精心装修，家私家电齐全，', '价格_详情': ' 2300 元/月（押二付一）   拎包入住  随时看房  ', '价格': '2300', '出租方式': '整租', '朝向': '南北', '户型': '3室2厅1卫', '楼层': '中层 楼层（共25层）', '建筑面积': '98平米', '装修': '精装修', '小区': ['翡翠绿洲森林半岛', '增城', '广园东'], '地址': ['开创大道北埔南路'], '网站': 'https://gz.zu.fang.com/chuzu/3_240746806_1.htm?channel=1,8'}
{'标题': '合生湖生花园精装2房，全新装修，钥匙在手随时看房', '价格_详情': ' 1800 元/月（押二付一） ', '价格': '1800', '出租方式': '整租', '朝向': '南', '户型': '2室2厅1卫', '楼层': '高层 楼层（共33层）', '建筑面积': '90平米', '装修': '精装修', '小区': ['合生湖山国际', '增城', '广园东'], '地址': ['陈家林路10号'], '网站': 'https://gz.zu.fang.com/chuzu/3_240747406_1.htm?channel=1,8'}
{'标题': '海伦堡花园 配套好 近地铁 望花园 业主急租，价格可谈', '价格_详情': ' 2300 元/月（押二付

TimeoutException: Message: 


# 番禺(超过100页)

## 一居(29页)

## 二居(54页)

## 三居(65页)

## 四居(21页)

## 四居以上(13页)

In [9]:
def parse_detail_page(source):
    Info = {}
    html = etree.HTML(source)
    Info['标题'] = html.xpath("//div[@class='tab-cont clearfix']/h1/text()")[0]
#     Info['价格'] = html.xpath("//div[@class='tr-line clearfix zf_new_title']//i/text()")[0] \
#                                 + html.xpath("//div[@class='tr-line clearfix zf_new_title']/div//text()")
    Info['价格_详情'] = html.xpath("//div[@class='tr-line clearfix zf_new_title']//text()")
    Info['价格_详情'] = " ".join(map(lambda x:x.strip(), Info['价格_详情']))
    Info['价格'] = html.xpath("//div[@class='tr-line clearfix zf_new_title']//i/text()")[0]
    Info['出租方式'] = html.xpath("//div[@class='trl-item1 w146']")[0].xpath("./div/text()")[0].strip()
    Info['朝向'] =     html.xpath("//div[@class='trl-item1 w146']")[1].xpath("./div[@class='tt']/text()")[0]
    Info['户型'] = html.xpath("//div[@class='trl-item1 w182']")[0].xpath("./div[@class='tt']/text()")[0]
    Info['楼层'] = html.xpath("//div[@class='trl-item1 w182']")[1].xpath("./div[@class='tt']/text()")[0] + " " + html.xpath("//div[@class='trl-item1 w182']")[1].xpath("./div[@class='font14']/text()")[0]
    Info['建筑面积'] = html.xpath("//div[@class='trl-item1 w132']")[0].xpath("./div[@class='tt']/text()")[0]
    Info['装修'] = html.xpath("//div[@class='trl-item1 w132']")[1].xpath("./div[@class='tt']/text()")[0]
    Info['小区'] = html.xpath("//div[contains(@class,'rcont')]")[0].xpath("./a/text()")
    Info['地址'] = html.xpath("//div[contains(@class,'rcont')]")[1].xpath("./a/text()")
#     Info['朝向'] = html.xpath("//div[@class='trl-item1 w146'][2]/div[@class='tt']/text()")[0]
#     Info['户型'] = html.xpath("//div[@class='trl-item1 w182']/div[@class='tt']/text()")[0]
#     Info['楼层'] = html.xpath("//div[@class='trl-item1 w182']/div/text()")
#     Info['建筑面积'] = html.xpath("//div[@class='trl-item1 w132'][1]/div[@class='tt']/text()")[0]
#     Info['装修'] = html.xpath("//div[@class='trl-item1 w132'][2]/div[@class='tt']/text()")[0]
    return Info

In [ ]:
PROJECT_PATH = os.getcwd()
PROJECT_PATH = os.path.join(PROJECT_PATH, "fang_zf", 'fang_zf.csv')

def __init__(self):
    self.json = open(PROJECT_PATH 'fang_zf.json')
    

In [43]:
PROJECT_PATH = os.getcwd()
PROJECT_PATH = os.path.join(PROJECT_PATH, "fang_zf")
f = open(os.path.join(PROJECT_PATH, 'fang_xf{}.json'.format(datetime.now().strftime('%Y-%m-%d'))), 'w', encoding='utf-8')
a = {'测试':'232', "测试2":'234324'}
jsonbj = json.dumps(a)
jsonbj = json.dumps(a)

f.write(jsonbj)
f.write('\n'+jsonbj)

f.close()